### Libraries

In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import re
import emoji

from decouple import config
from unicodedata import normalize

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/danipinho/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/danipinho/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Data

In [2]:
DATA_PATH = config("DATA_PATH")

In [3]:
data = pd.read_csv(DATA_PATH + "/Data/tweets_raw.csv", index_col=0)

### Options

In [4]:
pd.set_option("max_colwidth", None)

### Functions

In [5]:
# Remove emojis from tweets.
def delete_emoji(text):
    raw = emoji.demojize(text)
    plain_text = re.sub(r":[A-Za-zÀ-ü_'.-]+:", '', raw)
    return plain_text


In [6]:
# Remove Hyperlinks, Users mention, Hashtags, Line breaks, Portuguese diacritics characters, and decapitalize text.
def clean_text(plain_text):
    cleaned_text = re.sub(r" ?https?://[A-Za-z0-9.-_]+", '', plain_text)
    cleaned_text = re.sub(r"@[A-Za-z0-9_]{4,}", '', cleaned_text)
    cleaned_text = re.sub(r"#[A-Za-zÀ-ü0-9_]+", '', cleaned_text)
    cleaned_text = re.sub(r"[\n]+", ' ', cleaned_text)
    cleaned_text = normalize('NFKD', cleaned_text).encode('ASCII', 'ignore').decode('ASCII')
    return cleaned_text.lower()

In [7]:
# Remove base and query's stopwords 
def remove_stopwords(cleaned_text, words_list):
  extra_words = words_list
  stopwords = nltk.corpus.stopwords.words("portuguese")
  stopwords = [normalize('NFKD', word).encode('ASCII', 'ignore').decode('ASCII') for word in stopwords]
  stopwords.extend(extra_words)
  tokenizer = nltk.RegexpTokenizer(r"\w+")
  tokens = tokenizer.tokenize(cleaned_text)
  new_text = []
  for token in tokens:
    if len(token) > 2 and token not in stopwords:
      new_text.append(token)
  output = ' '.join(new_text)
  return output

In [8]:
# get words to include in the stopwords list
def get_stopwords(query):
    extra_words = ["nao", "pra", "que", "mto", "aqui", "vou", "vai", "pode", "tudo", "desde", "ser", "ter", "deve", "sobre", "tao", "eletrica", "eletrico"]
    query_token = nltk.word_tokenize(query.lower())
    extra_words.extend(query_token)
    return extra_words
    

In [9]:
# Pipeline for cleaning tweet.
def preprocessing_pipeline(query, text):
    plain_text = delete_emoji(text)
    cleaned_text = clean_text(plain_text)
    final_text = remove_stopwords(cleaned_text, get_stopwords(query))
    return final_text


### Code

In [10]:
data["clean_text"] = data.apply(lambda row: preprocessing_pipeline(row["query"], row["text"]), axis=1)

In [11]:
data["text_token"] = data["clean_text"].apply(lambda x: nltk.word_tokenize(x))

In [12]:
data[["query", "text", "clean_text", "text_token"]].head()

,query,text,clean_text,text_token
0,Operador Nacional do Sistema Eletrico,"O ONS - Operador Nacional do Sistema Elétrico e a Chesf iniciaram imediatamente a retomada das cargas para a concessionária e o serviço foi normalizado gradativamente. Pouco tempo depois, todos os clientes tiveram o fornecimento regularizado.",ons chesf iniciaram imediatamente retomada cargas concessionaria servico normalizado gradativamente pouco tempo todos clientes fornecimento regularizado,"[ons, chesf, iniciaram, imediatamente, retomada, cargas, concessionaria, servico, normalizado, gradativamente, pouco, tempo, todos, clientes, fornecimento, regularizado]"
1,Operador Nacional do Sistema Eletrico,"O Nordeste registrou no domingo um recorde de geração de energia solar fotovoltaica instantânea, com 1,16 gigawatts (GW) gerados às 10h45, informou nesta segunda-feira o Operador Nacional do Sistema Elétrico (ONS). https://t.co/vjnTpvclWz",nordeste registrou domingo recorde geracao energia solar fotovoltaica instantanea gigawatts gerados 10h45 informou nesta segunda feira ons,"[nordeste, registrou, domingo, recorde, geracao, energia, solar, fotovoltaica, instantanea, gigawatts, gerados, 10h45, informou, nesta, segunda, feira, ons]"
2,Operador Nacional do Sistema Eletrico,"É melhor o Operador Nacional do Sistema Elétrico aumentar a produção hoje, pq tende a ficar instável com tanto secador ligado...",melhor aumentar producao hoje tende ficar instavel tanto secador ligado,"[melhor, aumentar, producao, hoje, tende, ficar, instavel, tanto, secador, ligado]"
3,Operador Nacional do Sistema Eletrico,Visita Técnica ao Operador Nacional do Sistema Elétrico (ONS) em Brasília-DF! #IFG #IFGCampusGoiânia #ONS #GynBot\n👨🏻‍🏫👷🏻‍♀👷🏻⚡⚡⚡🤖👏🏻🖖🏻🙏🏻🤓 @ Operador Nacional Do Sistema ONS https://t.co/dhCeEO8Ey0,visita tecnica ons brasilia ons,"[visita, tecnica, ons, brasilia, ons]"
4,Operador Nacional do Sistema Eletrico,"New post in Forex Consultor Expert BR News: Money Times\nONS eleva previsão de chuva para hidrelétricas do Sudeste no mês\n\nO Operador Nacional do Sistema Elétrico (ONS) elevou nesta sexta-feira a previsão de chuvas para a região Sudeste, onde estão as pri… https://t.co/Qu3d5LLC1H",new post forex consultor expert news money times ons eleva previsao chuva hidreletricas sudeste mes ons elevou nesta sexta feira previsao chuvas regiao sudeste onde pri,"[new, post, forex, consultor, expert, news, money, times, ons, eleva, previsao, chuva, hidreletricas, sudeste, mes, ons, elevou, nesta, sexta, feira, previsao, chuvas, regiao, sudeste, onde, pri]"


In [13]:
tweets_preprocessed = data[["query", "text", "clean_text", "text_token"]]

In [14]:
tweets_preprocessed.to_json(DATA_PATH + "/Data/tweets_preprocessed_json.json")

In [15]:
tweets_preprocessed.to_csv(DATA_PATH + "/Data/tweets_preprocessed.csv")